# Week 2: Data Manipulation and Exploratory Data Analysis

## Learning Objectives
By the end of this notebook, you will be able to:
- Master advanced Pandas operations for data manipulation
- Perform comprehensive exploratory data analysis (EDA)
- Handle missing data and outliers effectively
- Create meaningful visualizations with Matplotlib and Seaborn
- Apply data cleaning techniques to real datasets

---

## 1. Setup and Data Loading

Let's start by importing our libraries and creating a realistic dataset for analysis.

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

In [ ]:
# Create a realistic customer dataset
np.random.seed(42)
n_customers = 1000

# Generate synthetic customer data with realistic relationships
ages = np.random.normal(35, 12, n_customers)
ages = np.clip(ages, 18, 80).astype(int)

# Income correlated with age
base_income = 30000 + (ages - 18) * 1200 + np.random.normal(0, 15000, n_customers)
income = np.clip(base_income, 20000, 150000)

# Purchase behavior
purchase_frequency = np.random.poisson(8, n_customers)
avg_purchase_amount = 50 + (income / 1000) * 0.5 + np.random.normal(0, 20, n_customers)
avg_purchase_amount = np.clip(avg_purchase_amount, 10, 500)

# Customer satisfaction
satisfaction = 3 + (income / 50000) + (avg_purchase_amount / 100) + np.random.normal(0, 0.8, n_customers)
satisfaction = np.clip(satisfaction, 1, 5)

# Categories and regions
categories = ['Electronics', 'Clothing', 'Books', 'Home & Garden', 'Sports']
regions = ['North', 'South', 'East', 'West', 'Central']
genders = ['Male', 'Female', 'Other']

# Create the dataset
customer_data = {
    'customer_id': range(1, n_customers + 1),
    'age': ages,
    'gender': np.random.choice(genders, n_customers, p=[0.48, 0.48, 0.04]),
    'income': income,
    'region': np.random.choice(regions, n_customers),
    'preferred_category': np.random.choice(categories, n_customers),
    'purchase_frequency': purchase_frequency,
    'avg_purchase_amount': avg_purchase_amount,
    'customer_satisfaction': satisfaction,
    'years_as_customer': np.random.exponential(2, n_customers)
}

df = pd.DataFrame(customer_data)

# Round numerical columns
df['income'] = df['income'].round(0)
df['avg_purchase_amount'] = df['avg_purchase_amount'].round(2)
df['customer_satisfaction'] = df['customer_satisfaction'].round(1)
df['years_as_customer'] = df['years_as_customer'].round(1)

# Introduce missing values (realistic scenario)
missing_indices = np.random.choice(df.index, size=50, replace=False)
df.loc[missing_indices[:25], 'income'] = np.nan
df.loc[missing_indices[25:], 'customer_satisfaction'] = np.nan

print(f"Dataset created with {len(df)} customers")
print(f"Dataset shape: {df.shape}")

## 2. Initial Data Exploration

The first step in any data analysis is understanding what we're working with.

In [ ]:
# Basic dataset information
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n=== FIRST 5 ROWS ===")
print(df.head())

In [ ]:
# Data types and missing values
print("=== DATA TYPES AND MISSING VALUES ===")
print(df.info())

print("\n=== MISSING VALUES SUMMARY ===")
missing_summary = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing Percentage': (df.isnull().sum() / len(df)) * 100
})
print(missing_summary[missing_summary['Missing Count'] > 0])

In [ ]:
# Summary statistics
print("=== NUMERICAL VARIABLES SUMMARY ===")
print(df.describe())

print("\n=== CATEGORICAL VARIABLES SUMMARY ===")
categorical_cols = ['gender', 'region', 'preferred_category']
for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print(df[col].value_counts())

## 3. Advanced Pandas Operations

Let's explore powerful data manipulation techniques.

In [ ]:
# Advanced filtering and selection
print("=== ADVANCED FILTERING ===")

# High-value customers (top 10% by income)
income_threshold = df['income'].quantile(0.9)
high_value = df[df['income'] >= income_threshold]
print(f"High-value customers (income >= ${income_threshold:,.0f}): {len(high_value)}")

# Complex conditions
young_frequent = df[(df['age'] < 30) & (df['purchase_frequency'] > 10)]
print(f"Young frequent buyers: {len(young_frequent)}")

# Using isin() for multiple values
premium_regions = df[df['region'].isin(['North', 'South']) & (df['customer_satisfaction'] >= 4.0)]
print(f"Satisfied customers from North/South: {len(premium_regions)}")

In [ ]:
# Groupby operations and aggregations
print("=== GROUPBY ANALYSIS ===")

# Multiple aggregations by region
region_stats = df.groupby('region').agg({
    'age': ['mean', 'std'],
    'income': ['mean', 'median'],
    'purchase_frequency': 'mean',
    'customer_satisfaction': ['mean', 'count']
}).round(2)

print("Regional statistics:")
print(region_stats)

# Cross-tabulation
print("\nGender vs Category cross-tabulation:")
crosstab = pd.crosstab(df['gender'], df['preferred_category'], margins=True)
print(crosstab)

In [ ]:
# Feature engineering
print("=== FEATURE ENGINEERING ===")

# Create age groups
df['age_group'] = pd.cut(df['age'], 
                        bins=[0, 25, 35, 50, 100], 
                        labels=['18-25', '26-35', '36-50', '50+'])

# Income categories
df['income_category'] = pd.cut(df['income'], 
                              bins=[0, 40000, 70000, 100000, float('inf')], 
                              labels=['Low', 'Medium', 'High', 'Very High'])

# Calculated fields
df['monthly_spending'] = df['purchase_frequency'] * df['avg_purchase_amount']
df['spending_per_year'] = df['monthly_spending'] * 12

# Customer value score (composite metric)
df['customer_value_score'] = (
    (df['monthly_spending'] / df['monthly_spending'].max()) * 0.4 +
    (df['customer_satisfaction'] / 5) * 0.3 +
    (df['years_as_customer'] / df['years_as_customer'].max()) * 0.3
) * 100

print("New features created:")
print(f"- Age groups: {df['age_group'].value_counts().to_dict()}")
print(f"- Income categories: {df['income_category'].value_counts().to_dict()}")
print(f"- Average monthly spending: ${df['monthly_spending'].mean():.2f}")

## 4. Handling Missing Data

Missing data is common in real datasets. Let's explore different strategies.

In [ ]:
# Visualize missing data patterns
print("=== MISSING DATA VISUALIZATION ===")

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Missing data heatmap
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis', ax=axes[0])
axes[0].set_title('Missing Data Pattern')

# Missing data counts
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]
axes[1].bar(missing_counts.index, missing_counts.values, color='coral')
axes[1].set_title('Missing Data Count by Column')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Total missing values: {df.isnull().sum().sum()}")

In [ ]:
# Different missing data strategies
print("=== MISSING DATA STRATEGIES ===")

# Strategy 1: Simple imputation
df_simple = df.copy()
df_simple['income'].fillna(df_simple['income'].median(), inplace=True)
df_simple['customer_satisfaction'].fillna(df_simple['customer_satisfaction'].mean(), inplace=True)

print(f"Simple imputation - remaining missing values: {df_simple.isnull().sum().sum()}")

# Strategy 2: Group-based imputation
df_advanced = df.copy()

# Fill income based on age group and region
for age_grp in df_advanced['age_group'].unique():
    if pd.isna(age_grp):
        continue
    for region in df_advanced['region'].unique():
        mask = (df_advanced['age_group'] == age_grp) & (df_advanced['region'] == region)
        group_median = df_advanced.loc[mask, 'income'].median()
        
        if not pd.isna(group_median):
            df_advanced.loc[mask & df_advanced['income'].isna(), 'income'] = group_median

# Fill remaining with overall median
df_advanced['income'].fillna(df_advanced['income'].median(), inplace=True)
df_advanced['customer_satisfaction'].fillna(df_advanced['customer_satisfaction'].mean(), inplace=True)

print(f"Advanced imputation - remaining missing values: {df_advanced.isnull().sum().sum()}")

# Use the advanced imputed dataset
df = df_advanced.copy()
print("Using advanced imputation for further analysis.")

## 5. Outlier Detection and Treatment

Outliers can significantly impact analysis. Let's identify and handle them.

In [ ]:
# Visualize outliers with box plots
print("=== OUTLIER VISUALIZATION ===")

numerical_cols = ['age', 'income', 'purchase_frequency', 'avg_purchase_amount', 'customer_satisfaction']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, col in enumerate(numerical_cols):
    axes[i].boxplot(df[col].dropna())
    axes[i].set_title(f'{col} - Box Plot')
    axes[i].set_ylabel(col)

# Remove empty subplot
fig.delaxes(axes[5])
plt.tight_layout()
plt.show()

In [ ]:
# Statistical outlier detection
def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("=== OUTLIER ANALYSIS ===")
for col in ['income', 'purchase_frequency', 'avg_purchase_amount']:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    print(f"\n{col.upper()}:")
    print(f"  Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    print(f"  Normal range: [{lower:.2f}, {upper:.2f}]")

## 6. Comprehensive EDA with Visualizations

Now let's create comprehensive visualizations to understand our data.

In [ ]:
# Distribution analysis
print("=== DISTRIBUTION ANALYSIS ===")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Age distribution
axes[0, 0].hist(df['age'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Age Distribution')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')

# Income distribution
axes[0, 1].hist(df['income'], bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('Income Distribution')
axes[0, 1].set_xlabel('Income ($)')
axes[0, 1].set_ylabel('Frequency')

# Purchase frequency
axes[0, 2].hist(df['purchase_frequency'], bins=20, alpha=0.7, color='coral', edgecolor='black')
axes[0, 2].set_title('Purchase Frequency Distribution')
axes[0, 2].set_xlabel('Purchases per Month')
axes[0, 2].set_ylabel('Frequency')

# Customer satisfaction
axes[1, 0].hist(df['customer_satisfaction'], bins=20, alpha=0.7, color='gold', edgecolor='black')
axes[1, 0].set_title('Customer Satisfaction Distribution')
axes[1, 0].set_xlabel('Satisfaction Score')
axes[1, 0].set_ylabel('Frequency')

# Monthly spending
axes[1, 1].hist(df['monthly_spending'], bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1, 1].set_title('Monthly Spending Distribution')
axes[1, 1].set_xlabel('Monthly Spending ($)')
axes[1, 1].set_ylabel('Frequency')

# Customer value score
axes[1, 2].hist(df['customer_value_score'], bins=30, alpha=0.7, color='pink', edgecolor='black')
axes[1, 2].set_title('Customer Value Score Distribution')
axes[1, 2].set_xlabel('Value Score')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Categorical analysis
print("=== CATEGORICAL ANALYSIS ===")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gender distribution
gender_counts = df['gender'].value_counts()
axes[0, 0].pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('Gender Distribution')

# Region distribution
region_counts = df['region'].value_counts()
axes[0, 1].bar(region_counts.index, region_counts.values, color='lightblue')
axes[0, 1].set_title('Customer Distribution by Region')
axes[0, 1].set_xlabel('Region')
axes[0, 1].set_ylabel('Number of Customers')

# Preferred category
category_counts = df['preferred_category'].value_counts()
axes[1, 0].bar(category_counts.index, category_counts.values, color='lightcoral')
axes[1, 0].set_title('Preferred Category Distribution')
axes[1, 0].set_xlabel('Category')
axes[1, 0].set_ylabel('Number of Customers')
axes[1, 0].tick_params(axis='x', rotation=45)

# Age group distribution
age_group_counts = df['age_group'].value_counts()
axes[1, 1].bar(age_group_counts.index, age_group_counts.values, color='gold')
axes[1, 1].set_title('Age Group Distribution')
axes[1, 1].set_xlabel('Age Group')
axes[1, 1].set_ylabel('Number of Customers')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
print("=== CORRELATION ANALYSIS ===")

# Select numerical columns for correlation
numerical_cols = ['age', 'income', 'purchase_frequency', 'avg_purchase_amount', 
                 'customer_satisfaction', 'years_as_customer', 'monthly_spending', 'customer_value_score']

correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix of Numerical Variables')
plt.tight_layout()
plt.show()

# Print strong correlations
print("\nStrong correlations (|r| > 0.5):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:
            print(f"{correlation_matrix.columns[i]} vs {correlation_matrix.columns[j]}: {corr_val:.3f}")

## 7. Practice Exercises

Now it's your turn to practice what you've learned!

### Exercise 1: Advanced Filtering
Create filters to find:
1. Customers aged 25-40 with high satisfaction (>4.0)
2. Electronics customers with monthly spending > $500
3. Female customers from East or West regions

In [ ]:
# Your code here
# Exercise 1 solutions

# 1. Customers aged 25-40 with high satisfaction
satisfied_middle_age = df[(df['age'] >= 25) & (df['age'] <= 40) & (df['customer_satisfaction'] > 4.0)]
print(f"Satisfied middle-aged customers: {len(satisfied_middle_age)}")

# 2. Electronics customers with high monthly spending
high_spending_electronics = df[(df['preferred_category'] == 'Electronics') & (df['monthly_spending'] > 500)]
print(f"High-spending electronics customers: {len(high_spending_electronics)}")

# 3. Female customers from East or West
female_east_west = df[(df['gender'] == 'Female') & (df['region'].isin(['East', 'West']))]
print(f"Female customers from East/West: {len(female_east_west)}")

### Exercise 2: Groupby Analysis
Perform the following analyses:
1. Average income by age group and gender
2. Total monthly spending by region and preferred category
3. Customer satisfaction statistics by income category

In [ ]:
# Your code here
# Exercise 2 solutions

# 1. Average income by age group and gender
income_by_age_gender = df.groupby(['age_group', 'gender'])['income'].mean().round(2)
print("Average income by age group and gender:")
print(income_by_age_gender)

# 2. Total monthly spending by region and category
spending_by_region_category = df.groupby(['region', 'preferred_category'])['monthly_spending'].sum().round(2)
print("\nTotal monthly spending by region and category:")
print(spending_by_region_category)

# 3. Customer satisfaction by income category
satisfaction_by_income = df.groupby('income_category')['customer_satisfaction'].agg(['mean', 'std', 'count']).round(2)
print("\nCustomer satisfaction by income category:")
print(satisfaction_by_income)

### Exercise 3: Data Visualization Challenge
Create visualizations to answer these questions:
1. How does monthly spending vary across different age groups?
2. Which regions have the highest customer satisfaction?
3. Is there a relationship between income and years as customer?

In [ ]:
# Your code here
# Exercise 3 solutions

# 1. Monthly spending by age group
plt.figure(figsize=(10, 6))
df.boxplot(column='monthly_spending', by='age_group', ax=plt.gca())
plt.title('Monthly Spending Distribution by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Monthly Spending ($)')
plt.show()

# 2. Customer satisfaction by region
plt.figure(figsize=(10, 6))
satisfaction_by_region = df.groupby('region')['customer_satisfaction'].mean().sort_values(ascending=False)
satisfaction_by_region.plot(kind='bar', color='skyblue')
plt.title('Average Customer Satisfaction by Region')
plt.xlabel('Region')
plt.ylabel('Average Satisfaction Score')
plt.xticks(rotation=45)
plt.show()

# 3. Income vs years as customer
plt.figure(figsize=(10, 6))
plt.scatter(df['years_as_customer'], df['income'], alpha=0.6)
plt.xlabel('Years as Customer')
plt.ylabel('Income ($)')
plt.title('Income vs Years as Customer')
plt.show()

# Calculate correlation
correlation = df['years_as_customer'].corr(df['income'])
print(f"Correlation between years as customer and income: {correlation:.3f}")

## Summary

In this notebook, we've covered essential data manipulation and exploratory data analysis techniques:

### Key Skills Learned:
1. **Advanced Pandas Operations**: Complex filtering, groupby operations, and feature engineering
2. **Missing Data Handling**: Multiple imputation strategies and visualization techniques
3. **Outlier Detection**: Statistical methods using IQR and visualization approaches
4. **Comprehensive EDA**: Distribution analysis, correlation analysis, and categorical data exploration
5. **Data Visualization**: Using Matplotlib and Seaborn for meaningful insights

### Best Practices:
- Always start with basic data exploration before diving into analysis
- Handle missing data thoughtfully with appropriate strategies
- Visualize data distributions and relationships to gain insights
- Document your findings and reasoning throughout the analysis
- Validate assumptions and check for data quality issues

These skills form the foundation for more advanced machine learning techniques we'll explore in upcoming weeks!